# Model training and hyperparameter tuning

O objetivo deste notebook é realizar testes de treino em diferentes modelos e diferentes hiperparâmetros

<u>Output</u>: Artefatos do modelo treinado

In [1]:
import sys
sys.path.append('..')

import pandas as pd
from src.data.make_dataset import data_preparation
from src.features.build_features import cbk_feature_engineering

## 1. Datasets

In [2]:
train = pd.read_excel('../data/raw/Missão_Stone_-_Dados_de_trx_(3).xlsx', engine='openpyxl', sheet_name='Aba 1')
test = pd.read_excel('../data/raw/Missão_Stone_-_Dados_de_trx_(3).xlsx', engine='openpyxl', sheet_name='Aba 2')

In [3]:
train.head(3)

,Dia,Hora,Valor,Cartão,CBK
0,2015-05-01 00:00:00,00:01:54,36.54,536518******2108,Não
1,2015-05-01 00:00:00,00:03:46,36.54,536518******2108,Não
2,2015-05-01 00:00:00,00:08:50,69,453211******1239,Não


In [4]:
test.head(3)

,Dia,Hora,Valor,Cartão,CBK
0,2015-06-01,00:02:25,112.00,541555******5965,NaN
1,2015-06-01,00:30:45,112.00,406669******7350,NaN
2,2015-06-01,00:43:20,18.34,541187******4535,NaN


#### 1.1 Data prep

Nota: Foram criados utils em src para reproduzir todas as transformações e feature engineering performadas em notebook por reprodutibilidade

In [5]:
def cbk_data_prep(df, test=False):
    df = data_preparation(df, test)
    df = cbk_feature_engineering(df)
    return df

In [6]:
processed_train = cbk_data_prep(train)
processed_test = cbk_data_prep(test, test=True)

..\src\data\make_dataset.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['transaction_timestamp'] = df['Dia'] + df['Hora']


In [7]:
processed_train.head(3)

,Valor,CBK,periodo_do_dia,dia_da_semana,emissor_cartao
0,36.54,0,madrugada,Sexta-feira,36518
1,36.54,0,madrugada,Sexta-feira,36518
2,69.00,0,madrugada,Sexta-feira,53211


In [8]:
processed_test.head(3)

,Valor,CBK,periodo_do_dia,dia_da_semana,emissor_cartao
0,112.00,NaN,madrugada,Segunda-feira,41555
1,112.00,NaN,madrugada,Segunda-feira,06669
2,18.34,NaN,madrugada,Segunda-feira,41187


## 2. Model testing

# Todo: traintest splits